In [1]:
owner = 'test-18'
!pip install imbalanced-learn

  Using cached imbalanced_learn-0.12.3-py3-none-any.whl.metadata (8.3 kB)
Using cached imbalanced_learn-0.12.3-py3-none-any.whl (258 kB)


In [3]:
import polaris as po
import datamol as dm
import numpy as np

/home/ubuntu/micromamba/envs/moml-ilya/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
benchmark = po.load_benchmark("polaris/pkis1-kit-wt-mut-c-1")
train, test = benchmark.get_train_test_split(featurization_fn=dm.to_fp)
#train, test = benchmark.get_train_test_split()

2024-06-20 21:43:13.335 | INFO     | polaris._artifact:_validate_version:66 - The version of Polaris that was used to create the artifact (0.0.0) is different from the currently installed version of Polaris (dev).
2024-06-20 21:43:13.341 | INFO     | polaris._artifact:_validate_version:66 - The version of Polaris that was used to create the artifact (0.0.0) is different from the currently installed version of Polaris (dev).


In [5]:
ys = train.y
ys = np.stack([ys[target] for target in benchmark.target_cols], axis=1)
mask = ~np.any(np.isnan(ys), axis=1)
mask.sum()

X_train = train.X[mask]
y_train = ys[mask]

In [6]:
import numpy as np
import pandas as pd
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split, cross_val_score
from bayes_opt import BayesianOptimization
from sklearn.svm import SVC, LinearSVC


def get_param(optimizer):
    best_params = optimizer.max['params']
    best_params['C'] = float(best_params['C'])
    best_params['gamma'] = int(best_params['gamma'])

    return best_params

In [23]:
def opt_target(target, param_bounds):
    if isinstance(target, int) is True:
        target = benchmark.target_cols[target]
    print(target)
    ys = train.y[target]
    y_train = ys[mask]

    print(y_train)
    
    
    def svm_cv(C, gamma):

        C = float(C)
        gamma = float(gamma)
    
        svm = SVC(C=C, gamma=gamma, kernel='rbf',probability=False)
        
        # Perform cross-validation

        cval = cross_val_score(svm, X_train, y_train, scoring='accuracy', cv=5)
    
        
        return cval.mean()
    
    
    
    optimizer = BayesianOptimization(
        f=svm_cv,
        pbounds=param_bounds,
        verbose=2
    )
    optimizer.maximize(init_points=10, n_iter=30)
    best_params = get_param(optimizer)

    best_svm = SVC(
        C=best_params['C'],
        gamma=best_params['gamma'],
        random_state=42
    )
    
    best_svm.fit(X_train, y_train)

    return best_svm, best_params


In [24]:
param_bounds = {
    'C': (1e-3, 1e3),
    'gamma': (1e-4, 1e1)
}

models = {target: opt_target(target, param_bounds = param_bounds) for target in benchmark.target_cols}

CLASS_KIT_(T6701_mutant)
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0.
 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
|   iter    |  target   |     C     |   gamma   |
-------------------------------------------------
| 1         | 0.9747    | 729.0   

In [26]:
y_prob_ind = {target: model.decision_function(test.X) for target, (model, params) in models.items()}
y_pred_ind = {target: model.predict(test.X) for target, (model, params) in models.items()}

best_params_ind = {target: params for target, (model, params) in models.items()}

results_ind = benchmark.evaluate(y_pred=y_pred_ind, y_prob=y_prob_ind)

results_ind.name = "svm_ind_no augm"
results_ind.description = best_params_ind
results_ind.to_json('svm_ind.json')

/home/ubuntu/micromamba/envs/moml-ilya/lib/python3.11/site-packages/pydantic/main.py:347: UserWarning: Pydantic serializer warnings:
  Expected `str` but got `dict` - serialized value may not be as expected
  return self.__pydantic_serializer__.to_python(


In [48]:
models['CLASS_KIT'][0].decision_function(test.X)

array([-0.78021978, -0.78021978, -0.78021978, -0.78021978, -0.78021978,
       -0.78021978, -0.78021978, -0.78021978, -0.78021978, -0.78021978,
       -0.78021978, -0.78021978, -0.78021978, -0.78021978, -0.78021978,
       -0.78021978, -0.78021978, -0.78021978, -0.78021978, -0.78021978,
       -0.78021978, -0.78021978, -0.78021978, -0.78021978, -0.78021978,
       -0.78021978, -0.78021978, -0.78021978, -0.78021978, -0.78021978,
       -0.78021978, -0.78021978, -0.78021978, -0.78021978, -0.78021978,
       -0.78021978, -0.78021978, -0.78021978, -0.78021978, -0.78021978,
       -0.78021978, -0.78021978, -0.78021978, -0.78021978, -0.78021978,
       -0.78021978, -0.78021978, -0.78021978, -0.78021978, -0.78021978,
       -0.78021978, -0.78021978, -0.78021978, -0.78021978, -0.78021978,
       -0.78021978, -0.78021978, -0.78021978, -0.78021978, -0.78021978,
       -0.78021978, -0.78021978, -0.78021978, -0.78021978, -0.78021978,
       -0.78021978, -0.78021978, -0.78021978, -0.78021978, -0.78

In [27]:
results_ind

{
  "name": "svm_ind_no augm",
  "description": {
    "CLASS_KIT_(T6701_mutant)": {
      "C": 728.9973388298779,
      "gamma": 2
    },
    "CLASS_KIT_(V560G_mutant)": {
      "C": 248.13272148972155,
      "gamma": 8
    },
    "CLASS_KIT": {
      "C": 523.2794904458099,
      "gamma": 4
    }
  },
  "tags": [],
  "user_attributes": {},
  "owner": null,
  "polaris_version": "dev",
  "benchmark_name": "pkis1-kit-wt-mut-c-1",
  "benchmark_owner": {
    "slug": "polaris",
    "external_id": "org_2gtoaJIVrgRqiIR8Qm5BnpFCbxu",
    "type": "organization"
  },
  "github_url": null,
  "paper_url": null,
  "contributors": null,
  "artifact_id": null,
  "benchmark_artifact_id": "polaris/pkis1-kit-wt-mut-c-1",
  "results": [
    {
      "Test set": "test",
      "Target label": "CLASS_KIT_(T6701_mutant)",
      "Metric": "accuracy",
      "Score": 0.8390804598
    },
    {
      "Test set": "test",
      "Target label": "CLASS_KIT_(V560G_mutant)",
      "Metric": "accuracy",
      "Score": 0.8620689655
    },
    {
      "Test set": "test",
      "Target label": "CLASS_KIT",
      "Metric": "accuracy",
      "Score": 0.6206896552
    },
    {
      "Test set": "test",
      "Target label": "CLASS_KIT_(T6701_mutant)",
      "Metric": "f1",
      "Score": 0.0
    },
    {
      "Test set": "test",
      "Target label": "CLASS_KIT_(V560G_mutant)",
      "Metric": "f1",
      "Score": 0.0
    },
    {
      "Test set": "test",
      "Target label": "CLASS_KIT",
      "Metric": "f1",
      "Score": 0.0
    },
    {
      "Test set": "test",
      "Target label": "CLASS_KIT_(T6701_mutant)",
      "Metric": "roc_auc",
      "Score": 0.5136986301
    },
    {
      "Test set": "test",
      "Target label": "CLASS_KIT_(V560G_mutant)",
      "Metric": "roc_auc",
      "Score": 0.5
    },
    {
      "Test set": "test",
      "Target label": "CLASS_KIT",
      "Metric": "roc_auc",
      "Score": 0.5
    },
    {
      "Test set": "test",
      "Target label": "CLASS_KIT_(T6701_mutant)",
      "Metric": "pr_auc",
      "Score": 0.1647058824
    },
    {
      "Test set": "test",
      "Target label": "CLASS_KIT_(V560G_mutant)",
      "Metric": "pr_auc",
      "Score": 0.1379310345
    },
    {
      "Test set": "test",
      "Target label": "CLASS_KIT",
      "Metric": "pr_auc",
      "Score": 0.3793103448
    },
    {
      "Test set": "test",
      "Target label": "CLASS_KIT_(T6701_mutant)",
      "Metric": "mcc",
      "Score": 0.0
    },
    {
      "Test set": "test",
      "Target label": "CLASS_KIT_(V560G_mutant)",
      "Metric": "mcc",
      "Score": 0.0
    },
    {
      "Test set": "test",
      "Target label": "CLASS_KIT",
      "Metric": "mcc",
      "Score": 0.0
    },
    {
      "Test set": "test",
      "Target label": "CLASS_KIT_(T6701_mutant)",
      "Metric": "cohen_kappa",
      "Score": 0.0
    },
    {
      "Test set": "test",
      "Target label": "CLASS_KIT_(V560G_mutant)",
      "Metric": "cohen_kappa",
      "Score": 0.0
    },
    {
      "Test set": "test",
      "Target label": "CLASS_KIT",
      "Metric": "cohen_kappa",
      "Score": 0.0
    }
  ]
}